In [ ]:
# compute complexity scores
import pickle
from src.evaluation import compute_complexity_score, compute_batch_complexity_score


# read pkl data from file
path = 'public/simple/'

# read file
with open(path + 'masks_2.pkl', 'rb') as f:
    data = pickle.load(f)


In [ ]:
complexity_inst = compute_complexity_score(data)
print(f"Complexity per instance: {complexity_inst}")

complexity_batch = compute_batch_complexity_score(data)
print(f"Complexity per batch: {complexity_batch}")

# am i getting 0 bc is it too simple??

/Users/alicia/Documents/Master/TFM/learnable-masks-explainability-time-series/.env/lib/python3.10/site-packages/quantus/metrics/complexity/complexity.py:254: RuntimeWarning: invalid value encountered in divide
  a = np.array(np.reshape(a, newshape), dtype=np.float64) / np.sum(np.abs(a))


Complexity per instance: 0.0
Complexity per batch: 0.0
